In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from pprint import pprint
import numpy as np
import json
import traceback
import os

In [11]:
mappings = json.load(open("data_class_mapping.json","r"))

In [12]:
def getData(file_name):
    path  = os.path.join("bank",file_name)
    if os.path.exists(path):
        data_frame = pd.read_csv(path, sep = ";")
    else: print "File does not exist !!"
    return data_frame


In [13]:
def findMapp(df):
    all_columns = list(df)
    for each_col in all_columns:
        if not each_col in mappings.keys():
            continue
        df[each_col] = list(map(lambda x : mappings[each_col][x], df[each_col]))
    return df

In [14]:
df = getData("bank.csv")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [15]:
dfm = findMapp(df)
dfm.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,2,0,2,0,1787,0,0,2,19,9,79,1,-1,0,0,0
1,33,11,0,1,0,4789,1,1,2,11,4,220,1,339,4,2,0
2,35,3,2,3,0,1350,1,0,2,16,3,185,1,330,1,2,0
3,30,3,0,3,0,1476,1,1,0,3,5,199,4,-1,0,0,0
4,59,7,0,1,0,0,1,0,0,5,4,226,1,-1,0,0,0


In [16]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dfm[dfm.columns[:-1]] = scaler.fit_transform(dfm[dfm.columns[:-1]])


## Train Decision Tree ::

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X = dfm.values[:,:-1]
Y = dfm.values[:,-1]

X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.3, random_state = 100)

clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=3, min_samples_leaf=5)
clf_gini.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [24]:
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100,
 max_depth=3, min_samples_leaf=5)
clf_entropy.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [25]:
y_pred_gini = clf_gini.predict(X_test)
y_pred_entropy = clf_entropy.predict(X_test)

print "Accuracy-gini ::", "{0:.2f}".format(accuracy_score(y_test,y_pred_gini)*100)+ "%"
print "Accuracy-entropy ::", "{0:.2f}".format(accuracy_score(y_test,y_pred_entropy)*100)+ "%"

Accuracy-gini :: 91.38%
Accuracy-entropy :: 91.53%


## Train Random Forest ::

In [26]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators = 37, random_state=50)
clf_rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=37, n_jobs=1,
            oob_score=False, random_state=50, verbose=0, warm_start=False)

In [27]:
y_pred_rf = clf_rf.predict(X_test)
print "Accuracy-random forest ::", "{0:.2f}".format(accuracy_score(y_test,y_pred_rf)*100)+ "%"

Accuracy-random forest :: 90.71%
